In [25]:
import pandas as pd
import re

# Função para processar cada linha
def process_line(line):
    # Regex para capturar a palavra, tipos e nível
    match = re.match(r"^([\w'-]+)\s([a-z.,\s]+)\s([A-C][1-2])$", line.strip())
    if match:
        word = match.group(1)
        types = [t.strip() for t in match.group(2).split(",")]  # Separa os tipos (ex.: "v., n.")
        level = match.group(3)
        return [{"Word": word, "Type": t, "Level": level} for t in types]
    return []

# Processar o arquivo
data = []
with open("words_oxford_pdf.txt", "r") as file:
    for line in file:
        if line.strip():  # Ignorar linhas vazias
            processed = process_line(line)
            data.extend(processed)  # Adicionar os registros processados

# Criar DataFrame
df = pd.DataFrame(data)

# Salvar em CSV para verificação futura
df.to_csv("words_oxford_levels.csv", index=False)

# Exibir os primeiros registros
WORDS = df[df["Word"]=="alien"].head(20)

,Word,Type,Level


In [87]:
from deep_translator import GoogleTranslator

# Função para traduzir sentenças
def translate_sentences(sentence, source_lang="en", target_lang="pt"):
    translator = GoogleTranslator(source=source_lang, target=target_lang)
    try:
        translation = translator.translate(sentence)
    except Exception as e:
        print(f"Erro ao traduzir a frase '{sentence}': {e}")
        translation = "Erro na tradução"
    return translation

WORDS = df.sample(100)["Word"].tolist()

In [88]:
import tkinter as tk
from tkinter import messagebox
import random


class HangmanGame:
    def __init__(self, root):
        self.root = root
        self.root.title("Hangman Game")

        self.word = random.choice(WORDS).upper()
        self.word_translate = translate_sentences(self.word)
        self.guessed_word = ["_"] * len(self.word)
        self.remaining_attempts = 6
        self.guessed_letters = set()

        self.setup_ui()

    def setup_ui(self):
        # Exibição da palavra
        self.word_label = tk.Label(self.root, text=" ".join(self.guessed_word), font=("Arial", 20))
        self.word_label.pack(pady=20)

        # Tentativas restantes
        self.attempts_label = tk.Label(self.root, text=f"Remaining Attempts: {self.remaining_attempts}", font=("Arial", 14))
        self.attempts_label.pack(pady=10)

        # Entrada da letra
        self.input_frame = tk.Frame(self.root)
        self.input_frame.pack(pady=10)

        tk.Label(self.input_frame, text="Enter a letter:", font=("Arial", 12)).pack(side="left", padx=5)
        self.letter_entry = tk.Entry(self.input_frame, width=5, font=("Arial", 14))
        self.letter_entry.pack(side="left", padx=5)
        self.guess_button = tk.Button(self.input_frame, text="Guess", command=self.check_guess)
        self.guess_button.pack(side="left", padx=5)

        # Letras já usadas
        self.guessed_label = tk.Label(self.root, text="Guessed Letters: ", font=("Arial", 12))
        self.guessed_label.pack(pady=10)

        # Dica
        
        self.word_translate_label = tk.Label(self.root, text=f"Word Translate: {self.word_translate.capitalize()} ", font=("Arial", 12))
        self.word_translate_label.pack(pady=10)

        

    def check_guess(self):
        # Obter entrada e limpar o campo
        letters = self.letter_entry.get().strip().upper()
        self.letter_entry.delete(0, tk.END)

        if not letters or not letters.isalpha():
            messagebox.showwarning("Invalid Input", "Please enter one or more valid letters!")
            return

        # Processar cada letra individualmente
        for letter in letters:
            if letter in self.guessed_letters:
                messagebox.showinfo("Repeated Letter", f"You've already guessed '{letter}'!")
                continue

            # Adicionar a letra às letras já tentadas
            self.guessed_letters.add(letter)
            self.update_guessed_label()

            # Verificar se a letra está na palavra
            if letter in self.word:
                self.update_word(letter)
            else:
                self.remaining_attempts -= 1
                self.attempts_label.config(text=f"Remaining Attempts: {self.remaining_attempts}")

            # Verificar estado do jogo após cada letra
            self.check_game_status()

        # Atualizar o estado do jogo após processar todas as letras
        self.check_game_status()


    def update_word(self, letter):
        for i, char in enumerate(self.word):
            if char == letter:
                self.guessed_word[i] = letter
        self.word_label.config(text=" ".join(self.guessed_word))

    def update_guessed_label(self):
        self.guessed_label.config(text=f"Guessed Letters: {', '.join(sorted(self.guessed_letters))}")
        self.word_translate_label.config(text=f"Word Translate: {self.word_translate}")


    def check_game_status(self):
        if "_" not in self.guessed_word:
            messagebox.showinfo("You Win!", f"Congratulations! You guessed the word: {self.word}")
            self.reset_game()
        elif self.remaining_attempts == 0:
            messagebox.showerror("Game Over", f"You lost! The word was: {self.word}")
            self.reset_game()

    def reset_game(self):
        self.word = random.choice(WORDS).upper()
        self.word_translate = translate_sentences(self.word)
        self.guessed_word = ["_"] * len(self.word)
        self.remaining_attempts = 6
        self.guessed_letters = set()

        self.word_label.config(text=" ".join(self.guessed_word))
        self.attempts_label.config(text=f"Remaining Attempts: {self.remaining_attempts}")
        self.guessed_label.config(text="Guessed Letters: ")
        self.word_translate_label.config(text=f"Word Translate: {self.word_translate}")

# Inicializa o jogo
if __name__ == "__main__":
    root = tk.Tk()
    game = HangmanGame(root)
    root.mainloop()
